In [1]:
#Dependancies 
import numpy as np 
import pandas as pd
#import seaborn as sns 
import cv2
import skimage.io as io
import keras
import os
import glob
#import matplotlib.pyplot as plt
#%matplotlib inline
#import plotly.offline as py
#import plotly.graph_objs as go
#import plotly.tools as tls 
#py.init_notebook_mode(connected=True)
import random 
import shutil

from skimage.io import imread, imshow
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

#from subprocess import check_output
#print(check_output(["ls", "~/KarlieRay/Practicum/train"]).decode("utf8"))

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from keras import backend as K
from skimage.transform import resize

Using TensorFlow backend.


In [4]:
#Split data into training and validation sets, keeping file organization. 
allFiles = glob.glob('/home/dldev/KarlieRay/Practicum/train/*/*')
print(allFiles[0])

split_point = int(round(0.7*len(allFiles)))
random.shuffle(allFiles)

train_list = allFiles[:split_point]
valid_list = allFiles[split_point:]
print('Train images: {}'.format(len(train_list)))
print('Validation images {}'. format(len(valid_list)))

# Divy them into their own directories.
train_data_path = '~/KarlieRay//Practicum/train_data'
validation_data_path = '~/KarlieRay//Practicum/valid_data'
if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
if not os.path.exists(validation_data_path):
    os.makedirs(validation_data_path)

for fpath in train_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([train_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))
    shutil.copyfile(fpath, '/'.join([train_data_path] +basename))
for fpath in valid_list:
    basename = fpath.split('/')[-2:]
    dest = '/'.join([validation_data_path] +basename)
    if not os.path.exists('/'.join(dest.split('/')[:-1])):
        os.makedirs('/'.join(dest.split('/')[:-1]))    
    shutil.copyfile(fpath, '/'.join([validation_data_path] +basename))

/home/dldev/KarlieRay/Practicum/train/Type_3/952.jpg
Train images: 1037
Validation images 444


In [17]:
K.set_image_dim_ordering('tf')

# Create numpy array
train_image_list = list()
train_label_list = list()
for image in train_list:
    im = cv2.imread(image)
    im = resize(im, (100, 100), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    train_label_list.append(im_type.split('Type_')[1])
    train_image_list.append(im)
print(train_label_list)
print(train_image_list)

# Reshape
#train_list = train_list.reshape(train_list[0], 100, 100, 3).astype('float32')
#valid_list = Reshape(valid_list[0], 100, 100, 3).astype('float32')

['2', '1', '3', '3', '3', '2', '3', '3', '2', '2', '2', '2', '2', '2', '3', '2', '2', '2', '3', '3', '1', '1', '1', '3', '2', '2', '3', '3', '3', '2', '2', '2', '3', '3', '3', '3', '2', '2', '2', '3', '3', '2', '3', '2', '2', '2', '1', '2', '2', '3', '2', '3', '2', '3', '3', '2', '3', '3', '2', '2', '2', '2', '3', '3', '2', '2', '2', '2', '2', '2', '3', '2', '3', '1', '3', '2', '3', '2', '2', '1', '3', '2', '2', '3', '3', '2', '3', '2', '2', '3', '3', '3', '2', '2', '2', '3', '2', '2', '2', '3', '3', '3', '3', '2', '3', '2', '3', '2', '2', '2', '1', '1', '2', '2', '2', '1', '2', '2', '3', '3', '3', '1', '3', '2', '3', '2', '1', '3', '2', '1', '3', '1', '3', '2', '2', '2', '3', '3', '1', '3', '2', '1', '2', '2', '2', '2', '2', '2', '3', '3', '3', '2', '1', '3', '3', '1', '1', '3', '1', '3', '2', '2', '3', '3', '2', '3', '2', '3', '3', '1', '1', '2', '2', '2', '2', '2', '3', '2', '2', '2', '3', '2', '1', '2', '2', '3', '2', '3', '3', '3', '2', '2', '2', '3', '2', '3', '3', '2', '2', '3',

In [18]:
test_image_list = list()
test_label_list = list()
for image in valid_list:
    im = cv2.imread(image)
    im = resize(im, (100, 100), mode='reflect').astype('float32')
    im_type = image.split('/')[-2]
    test_label_list.append(im_type.split('Type_')[1])
    test_image_list.append(im)
print(test_label_list)
print(test_image_list)

['2', '3', '3', '1', '2', '1', '2', '3', '2', '2', '1', '1', '2', '2', '2', '2', '3', '2', '3', '3', '2', '2', '3', '1', '2', '2', '1', '3', '3', '3', '3', '1', '2', '3', '3', '3', '2', '2', '1', '1', '1', '2', '3', '2', '2', '2', '2', '1', '2', '2', '3', '3', '1', '1', '2', '2', '1', '2', '2', '1', '3', '3', '1', '2', '2', '3', '2', '2', '3', '2', '3', '3', '3', '2', '2', '3', '2', '1', '2', '3', '2', '3', '3', '1', '2', '2', '2', '2', '2', '2', '3', '1', '2', '3', '1', '2', '1', '2', '2', '3', '2', '2', '2', '1', '3', '3', '2', '3', '2', '1', '1', '1', '1', '2', '2', '2', '1', '3', '3', '2', '2', '2', '1', '3', '2', '2', '3', '3', '3', '2', '3', '2', '3', '2', '2', '2', '2', '2', '2', '2', '2', '1', '3', '1', '2', '2', '3', '1', '3', '3', '2', '1', '3', '3', '3', '3', '1', '2', '2', '2', '3', '3', '2', '2', '2', '1', '3', '2', '3', '1', '1', '3', '2', '1', '2', '3', '2', '3', '1', '1', '3', '2', '1', '2', '2', '2', '2', '2', '2', '1', '3', '2', '2', '3', '2', '3', '2', '1', '3', '1',

In [19]:
print(im.shape)

(100, 100, 3)


In [20]:
X_train = np.array(train_image_list)
Y_train = np.array(train_label_list)

#dimensions of images
img_width, img_height = 100, 100

#one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
num_classes = Y_train.shape[1]

In [21]:
X_test = np.array(test_image_list)
Y_test = np.array(test_label_list)

#one hot encode outputs
Y_test = np_utils.to_categorical(Y_test)


In [22]:
# Simple CNN model.
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

def simple_cnn_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(Y_train.shape[1], activation = 'softmax'))
    
# Compile model
# Code modified from libphy https://github.com/libphy/dldev/blob/master/MNIST-Keras.ipynb

    model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy']) 
    return model

# Build model
model = simple_cnn_model()

In [23]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1037, 100, 100, 3)
(1037, 4)
(444, 100, 100, 3)
(444, 4)


In [24]:
# Fit the model
model.fit(X_train, Y_train, batch_size = 32, nb_epoch = 5, verbose = 1)
#for idx,im in enumerate(train_image_list):
#    model.fit(im, train_label_list[idx], batch_size=32, nb_epoch=5, verbose=1)

# Final evaluation of the model
#scores = model.evaluate(train_data_path, validation_data_path, verbose = 0)
scores = model.evaluate(X_test, Y_test, verbose = 0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Epoch 1/5
1037/1037 [==============================] - 1s - loss: 1.2727 - acc: 0.4687      